# LRCStats - Long Read Correction Statistics #

## Background ##
---
DNA sequencing is the process of constructing a string of characters taken from the set {A,C,G,T} (representing the nucleotide bases adenine, cytosine, guanine and thymine) to represent the order of nucleotides that compose a DNA molecule. Genomes, however, cannot be sequenced in full - only segments, or "reads" as we will call it, can be sequenced at a time. Currently, there are two commonly used sequencing technologies - Illumina cyclic reversible termination (CRT) systems, which output short, accurate reads and Pacific Biosciences single-molecule real-time (SMRT) technologies, which output long, inaccurate reads. (Goodwin et al, 2016) We will refer to reads outputted by Illumina CRT systems as "Illumina reads" and reads outputted by Pacific BioSciences SMRT systems as "PacBio reads".

Illumina short reads, as the name suggests, are short in length, with read lengths ranging from 100 to 150 base pairs (bp) depending on the system used. In contrast, the length of the _Eshercheria chia_ genome, a bacterial species with a relatively simple genome, is approximately 4 000 000 bp. (cite this) However, the error rate of Illumina reads is < 1% at most, with errors being mostly substitutions. (Goodwin et al, 2016) 

PacBio reads, on the other hand, are much longer than Illumina short reads, with mean lengths of 20kb for the Pacific BioSciences RS II system. However, the error rate profile is quite concerning - 13% single pass error rate with the majority of errors being insertions and deletions. (Goodwin et al, 2016)

Despite the low error rate of Illumina reads, the short lengths constrain the usage of Illumina short reads in genome assembly. This is because genomes commonly contain repeat sequences - regions of the genome that are similar, if not the same, to prior regions of the genome. If the input reads do not fully span the entire length of these repeats, this can cause the assembled genome to be fragmented, which is not ideal for downstream analysis. (cite this) The long lengths of PacBio reads are thus preferable for genome assembly as they can consolidate the majority of repeats in a genome and allow for longer, contiguous assembled genomes. However, the high error rate of PacBio reads can, as well, cause problems in downstream analysis.

### Long Read Correction Algorithms ###
Long read correction algorithms are a class of algorithms that attempt to reduce the error rate of long reads.
_Hybrid correction algorithms_ are a subset of these algorithms. The idea behind hybrid correction algorithms is to correct erroneous long reads with accurate short reads, thus creating long and highly accurate reads, which is ideal for genome assembly.

To correct long reads, recent hybrid correction algorithms follow two methods:

1. Map the short reads onto the long reads using a mapper, such as Burrows-Wheeler Aligner
2. Construct a de Bruijn graph using the short reads and align the long reads onto the graph structure

**Recently published hybrid correction algorithms**

| Name      | Date |
| :-------: | :--: |
| Proovread | 2014 |
| LoRDEC    | 2014 |
| Jabba     | 2016 |
| CoLoRMap  | 2016 |

## Methods ## 
---

![LRCStats pipeline](lrcstats_pipeline.png)

### Overview ###
Long Read Correction Statistics (LRCStats) is an open source pipeline that benchmarks long read correction algorithms using simulated long read data. The overall goal of LRCStats is to assess the accuracy of correction algorithms with simulated data where errors have been precisely identified. The current LRCStats pipeline is split into four distinct steps:

#### 1. Simulate Long and Short Reads ####
Currently, LRCStats uses SimLoRD and ART to simulate PacBio long and Illumina short reads, respectively. LRCStats is designed to be easily modified to use other long read simulators if the user so desires.

The long read simulator SimLoRD outputs simulated long reads in a FASTQ file format along with a SAM alignment file that specifies the true alignment of the simulated long reads with the reference genome. We then convert this SAM file into a [MAF](https://genome.ucsc.edu/FAQ/FAQformat.html#format5) file. Whereas SAM files represent the alignment of a long read and the reference sequence in the CIGAR format (cite this), MAF files directly represent the alignment of the two sequences. This allows us to align the corrected read onto the original alignment using a dynamic programming algorithm and perform statistics on this new three-way alignment.

#### 2. Correct Long Reads ####
LRCStats currently supports four hybrid correction algorithms: Proovread, LoRDEC, Jabba and CoLoRMap. The default parameters for each algorithm were chosen to best reflect common usage. LRCStats is designed to be easily modified to accomodate new long read correction algorithms, and not only hybrid correction algorithms.

#### 3. Construct Three-Way Alignments ####
Using the MAF file outputted in step 1, LRCStats aligns the corrected long read (cLR) onto the original reference (ref) and uncorrected long read alignment (uLR) to create a three-way alignment. This is done using a specially-designed dynamic programming algorithm that takes into account the structure of the corrected long reads outputted by the four hybrid correction algorithms in use.

The hybrid correction algorithms can fail to correct certain segments of the simulated long reads. Each algorithm handles this differently. CoLoRMap and LoRDEC indicate these uncorrected segments in the corrected long reads by outputting uncorrected bases in lowercase, whereas corrected bases are outputted in uppercase. Proovread "trims" these uncorrected segments from the corrected long reads to output only the corrected segments of the original uncorrected read. Jabba does not currently keep note of which bases in the corrected long read have been corrected, and may split the read into separate subreads. The alignment algorithm takes into account both cases when creating the three-way alignment.

#### 4. Collect Statistics on the Three-Way Alignments ####
The three-way alignment outputted in the previous step allows us to collect statistics in two different ways:

1. On the entire length of the read
2. On only those segments of the reads that have been corrected

This provides us a general method to compare the accuracy of corrected long reads outputted by the various algorithms. The statistics currently outputted by LRCStats are:

1. Error rate over all bases
2. Throughput, i.e. total number of outputted bases
3. Number of correct and incorrect bases
4. Number of deletions, insertions and substitutions among the incorrect bases

The above statistics are collected for both corrected and uncorrected reads. As well, these statistics are categorized into two other separate categories, which are: 

1. Statistics collected on the entire length of the three-way alignment
2. Statistics collected on only those segments of the corrected read which have been corrected and the corresponding segment of the uncorrected read

We call the former category the "untrimmed" statistics and the latter the "trimmed" statistics.

### Implementation Details ###

#### Programming Languages Used ####
Except for the aliger program, LRCStats is written in Python 2.7 for ease of maintenance and usage for the user. The aligner program is written in C++ 11 to take advantage of the speed of this compiled programming language. This is especially important for sequence alignments which can require a significant amount of time to compute despite having polynomial time complexity.

#### Code Structure ####
The code for this project is organized into the following main directories:

* `config`: contains the configuration files for different experiments
* `notebooks`: contains notebooks detailing the experiments featured in the LRCStats paper
* `pipeline`: contains Python modules for the construction of the PBS scripts for each experiment
* `scripts`: contains the scripts for each experiment
* `src`: contains Python scripts and C++ source code for processing, aligning and analyzing the corrected long reads

#### Alignment Algorithm ####
The sequence alignment algorithm used for the creation of the three-way alignments of the reference sequence, uncorrected and corrected long reads is based on the common dynamic programming algorithm for calculating the minimum edit distance between two sequences. We modified this algorithm in two ways:

1. Our algorithm constructs optimal local alignments by either:
  * anchoring uncorrected segments of corrected long reads to the corresponding identical segments in the uncorrected long reads if the corrected read is untrimmed.
  * allowing for unpenalized deletions between trimmed segments of corrected long reads if the corrected read is trimmed.
2. Our algorithm aligns a sequence to an alignment of two other sequences (i.e. we align the corrected long read with the _alignment_ of the uncorrected long read and the reference sequence).

Recall that the correction algorithms Jabba and Proovread output trimmed long reads, whereas CoLoRMap and LoRDEC output untrimmed long reads.

Given a corrected long read C[1..n] and an alignment A[1..m] of the reference sequence and uncorrected long read, the time complexity of this algorithm is then O(nm).

## Results ##
---

### Datasets ###
_Escherichia coli_, _Saccharomyces cerevisiae_, and _Drosophila melanogaster_ genomes were used as references for the short and long read simulators. These species were chosen due to their role as model organisms in the biological sciences - we have extensive knowledge concerning their genetic makeups. Thus, they serve as good bases to further develop our understanding of hybrid correction algorithms.

Links to download the reference genomes used in this study are featured in the following table:

| Organism                   | Source                                                 |
| :------------------------: | :----------------------------------------------------: |
| _Escherichia coli_         | https://www.ncbi.nlm.nih.gov/nuccore/NC_000913         |
| _Saccharomyces cerevisiae_ | http://www.yeastgenome.org/strain/S288C/overview       |
| _Drosophila melanogaster_  | http://www.fruitfly.org/sequence/release5genomic.shtml |

Empirical PacBio long read FASTQ files for _Escherichia coli_, _Saccharomyces cerevisiae_ and _Drosophila melanogaster_ were provided as input to SimLoRD to serve as sample distributions for long read lengths.

Ascension codes for the PacBio long read FASTQ files are given in the following table.

| Organism                   | Ascension Code(s)      |
| :------------------------: | :--------------------: |
| _Escherichia coli_         | SRR1284073             |
| _Saccharomyces cerevisiae_ | SRR1284074, SRR1284662 |
| _Drosophila melanogaster_  | SRR1204085             |

Note that the two FASTQ files for _S. cerevisiae_ were merged with a simple concatenation and were fed together into SimLoRD.

Illumina short read datasets were simulated with fold coverages of 50, 100 and 200. These coverages were chosen to reflect the coverages of common empirical Illumina short read datasets.

ART was used with the following parameters:
* read lengths were set to 100 bp
* mean fragment lengths were set to 300
* standard deviation of fragment lengths were set to 25

SimLoRD was used to simulate PacBio long read datasets with coverages of 10, 20, 50 and 75. Likewise with the selection of coverages for the simulated short reads, these were chosen to reflect the coverages of common empirical PacBio datasets. Empirical PacBio long read FASTQ files were inputted to SimLoRD to serve as sample distributions for read lengths. Note that SimLoRD does not have an option to input coverages directly - the number of reads required for a desired coverage given a reference genome and empirical PacBio long read FASTQ file must be calculated prior by the user. LRCStats provides a Python script that does precisely this given a reference genome, empirical long read FASTQ file and desired fold coverage. Additionally, the maximum number of passes for each read was set to 1 to maintain a high error rate in the simulated long reads. All other options were set to their defaults as of September 2016.

### Experiment Design ###
Proovread, LoRDEC, Jabba and CoLoRMap with and without the one-end anchor option were used to correct the long reads. All combinations of short and long read coverages were performed in the correction stage, except for those where the short read coverage is less than the long read coverage.

For LoRDeC, the following parameters were used:
- kmer length of 19
- error rate of 0.4
- number of target k-mers: 5
- max branch exploration of 200

Results are shown without performing `lordec-trim` on the corrected read datasets.

For Jabba, the following parameters were used:
- Short reads were pre-corrected using Karect
- The de Bruijn graph was created with Brownie with a k-mer length of 45 (graphCorrection mode)
- Thread usage of 6
- Minimum size of MEM of 20
- k-mer size of 45

For proovread, the long read quality offset value was set to 70 in order to avoid a bug that occurred occasionally when this option was not set. According to an issue page in the Proovread Github repository, this option has no effect on the correction process.

### Experiment Results ###

## Conclusion ##
---

## References ##
---

## Appendix ##
---

### LRCStats Manual ###
See it [here](README.md).

### Scripts Used in Featured Experiment ###

The following are the exact scripts used in the experiment featured in the results section. Note that shell variables corresponding to: 

1. the paths unique to the the cluster used for the experiment
2. specific experimental details (e.g. short and long read coverages, reference genome, etc.)

have been omitted.

#### Simulating Long and Short Reads using SimLoRD and ART ####

ART (Short Reads):
```bash
outputDir=$genomeDir/${experiment}/simulate/art/short-d${cov}
outputPrefix=$outputDir/${genome}-short-paired-d${cov}

mkdir -p $outputDir
$art -p -i $ref -l 100 -f $cov -m 300 -s 25 -o $outputPrefix

python $fq2fastq -i $outputDir

short1=${outputPrefix}1.fastq
short2=${outputPrefix}2.fastq
shortMerged=${outputPrefix}-merged.fastq

$merge_files shuffle -1 $short1 -2 $short2 -o $shortMerged
```

SimLoRD (Long Reads):
```bash
sam2maf=${lrcstats}/src/preprocessing/sam2maf/sam2maf.py
reads4coverage=${lrcstats}/src/preprocessing/reads4coverage.py
outputDir=${genomeDir}/${experiment}/simulate/simlord/long-d${cov}
outputPrefix=${outputDir}/${genome}-long-d${cov}

mkdir -p $outputDir
reads=$(python $reads4coverage -c $cov -i $fastq -r $ref)
$simlord -n $reads -sf $fastq -rr $ref $outputPrefix

sam=${outputPrefix}.fastq.sam
maf=${sam}

python $sam2maf -r $ref -s $sam -o $maf
```

#### Correcting Long Reads ####

CoLoRMap w/o OEA:
```bash
set -e
mkdir -p $outputDir

colormap=/home/seanla/Software/colormap/runCorr.sh
outputPrefix=$outputDir/$testName

cd $outputDir
$colormap $long $mergedShort $outputDir $outputPrefix ${PBS_NUM_PPN}
```

CoLoRMap w/ OEA:
```bash
set -e
mkdir -p $outputDir

cd $outputDir
${colormapOea} $long $mergedShort $outputDir $outputPrefix ${PBS_NUM_PPN}
```

Jabba, using Karect and Brownie:
```bash
set -e
mkdir -p $outputDir

karect=/home/seanla/Software/karect/karect
brownie=/home/seanla/Software/brownie/brownie
jabba=/home/seanla/Software/jabba/jabba
karectDir=$outputDir/karect
brownieDir=$outputDir/brownie
jabbaDir=$outputDir/jabba

mkdir -p $karectDir
mkdir -p $brownieDir
mkdir -p $jabbaDir

$karect -correct -inputfile=$short1 -inputfile=$short2 -resultdir=$karectDir -tempdir=$karectDir -celltype=haploid -matchtype=hamming -threads=${PBS_NUM_PPN}

short1=$karectDir/karect_${genome}-short-paired-d${shortCov}1.fastq
short2=$karectDir/karect_${genome}-short-paired-d${shortCov}2.fastq

$brownie graphCorrection -k 75 -p $brownieDir $short1 $short2

rm -r $karectDir

dbGraph=$brownieDir/DBGraph.fasta

$jabba -t ${PBS_NUM_PPN} -l 20 -k 75 -o $jabbaDir -g $dbGraph -fastq $long

rm -r $brownieDir
```

LoRDEC:
```bash
set -e
mkdir -p $outputDir

lordec=/home/seanla/Software/LoRDEC-0.6/lordec-correct
output=${outputDir}/${testName}.fasta
cd ${outputDir}
$lordec -T ${PBS_NUM_PPN} --trials 5 --branch 200 --errorrate 0.4 -2 ${short1} ${short2} -k 19 -s 3 -i ${long} -o ${output}
```

Proovread:
```bash
$proovread -t ${PBS_NUM_PPN} --lr-qv-offset 70 -s $short1 -s $short2 -l $long -p $output
```

#### Aligning Corrected Reads to Ref-uLR Alignment ####

CoLoRMap w/o OEA:
```bash
mkdir -p ${outputDir}

input=${inputDir}/${testName}_iter2.fasta

############### Sort FASTA ###########
echo 'Sorting FASTA file...'

sortfasta=${lrcstats}/src/preprocessing/sortfasta/sortfasta.py
sortedOutput=${outputDir}/sorted.fasta

python $sortfasta -i $input -o $sortedOutput

input=$sortedOutput

############### Prune the maf file(s) ###########
echo 'Pruning MAF file(s)...'

prunemaf=${lrcstats}/src/preprocessing/prunemaf/prunemaf.py
pruneOutput=${outputDir}/pruned
python $prunemaf -f $input -m $maf -o $pruneOutput

maf=${pruneOutput}.maf

############### Generate three-way alignment ###########
echo 'Generating three-way alignment...'
aligner=${lrcstats}/src/aligner/aligner
mafOutput=${outputDir}/${testName}.maf

$aligner maf -m $maf -c $input -o $mafOutput -p ${PBS_NUM_PPN}
```

CoLoRMap w/ OEA:
```bash
mkdir -p ${outputDir}

input=${inputDir}/${testName}_oea.fasta

############### Sort FASTA ###########
echo 'Sorting FASTA file...'

sortfasta=${lrcstats}/src/preprocessing/sortfasta/sortfasta.py
sortedOutput=${outputDir}/sorted.fasta

python $sortfasta -i $input -o $sortedOutput

input=$sortedOutput

############### Prune the maf file(s) ###########
echo 'Pruning MAF file(s)...'

prunemaf=${lrcstats}/src/preprocessing/prunemaf/prunemaf.py
pruneOutput=${outputDir}/pruned
python $prunemaf -f $input -m $maf -o $pruneOutput

maf=${pruneOutput}.maf

############### Generate three-way alignment ###########
echo 'Generating three-way alignment...'
aligner=${lrcstats}/src/aligner/aligner
mafOutput=${outputDir}/${testName}.maf

$aligner maf -m $maf -c $input -o $mafOutput -p ${PBS_NUM_PPN}
```

Jabba:
```bash
mkdir -p ${outputDir}

input=${inputDir}/jabba/Jabba-${genome}-long-d${longCov}.fastq

############### Convert FASTQ to FASTA ###########
echo 'Converting FASTQ to FASTA...'

fastq2fasta=${lrcstats}/src/preprocessing/fastq2fasta/fastq2fasta.py
outputq2a=$outputDir/${testName}

python $fastq2fasta -i $input -o $outputq2a

input=${outputq2a}.fasta

############### Processing Trimmed Reads ###########
echo 'Processing trimmed reads...'
concatenate=${lrcstats}/src/preprocessing/concatenate_trimmed/concatenate_trimmed.py
concatenated_output=${outputDir}/concatenated.fasta

python $concatenate -i $input -o ${concatenated_output}

input=${concatenated_output}

############### Prune the maf file(s) ###########
echo 'Pruning MAF file(s)...'

prunemaf=${lrcstats}/src/preprocessing/prunemaf/prunemaf.py
pruneOutput=${outputDir}/pruned
python $prunemaf -f $input -m $maf -o $pruneOutput

maf=${pruneOutput}.maf

############### Generate three-way alignment ###########
echo 'Generating three-way alignment...'
aligner=${lrcstats}/src/aligner/aligner
mafOutput=${outputDir}/${testName}.maf

$aligner maf -m $maf -c $input -t -o $mafOutput -p ${PBS_NUM_PPN}
```

LoRDEC:
```bash
set -e
mkdir -p ${outputDir}

input=${inputDir}/${testName}.fasta

############### Sort FASTA ###########
echo 'Sorting FASTA file...'

sortfasta=${lrcstats}/src/preprocessing/sortfasta/sortfasta.py
sortedOutput=${outputDir}/sorted.fasta

python $sortfasta -i $input -o $sortedOutput

input=$sortedOutput

############### Prune the maf file(s) ###########
echo 'Pruning MAF file(s)...'

prunemaf=${lrcstats}/src/preprocessing/prunemaf/prunemaf.py
pruneOutput=${outputDir}/pruned
python $prunemaf -f $input -m $maf -o $pruneOutput

maf=${pruneOutput}.maf

############### Generate three-way alignment ###########
echo 'Generating three-way alignment...'
aligner=${lrcstats}/src/aligner/aligner
mafOutput=${outputDir}/${testName}.maf

$aligner maf -m $maf -c $input -o $mafOutput -p ${PBS_NUM_PPN}
```

Proovread:
```bash
mkdir -p ${outputDir}

input=${inputDir}/${testName}.trimmed.fa

############### Processing Trimmed Reads ###########
echo 'Processing trimmed reads...'
concatenate=${lrcstats}/src/preprocessing/concatenate_trimmed/concatenate_trimmed.py
concatenated_output=${outputDir}/concatenated.fasta

python $concatenate -i $input -o ${concatenated_output}

input=${concatenated_output}

############### Prune the maf file(s) ###########
echo 'Pruning MAF file(s)...'

prunemaf=${lrcstats}/src/preprocessing/prunemaf/prunemaf.py
pruneOutput=${outputDir}/pruned
python $prunemaf -f $input -m $maf -o $pruneOutput

maf=${pruneOutput}.maf

############### Generate three-way alignment ###########
echo 'Generating three-way alignment...'
aligner=${lrcstats}/src/aligner/aligner
mafOutput=${outputDir}/${testName}.maf

$aligner maf -m $maf -c $input -t -o $mafOutput -p ${PBS_NUM_PPN}
```

#### Collection of Statistics ####

CoLoRMap w/ and w/o OEA:
```bash
set -e
mkdir -p ${outputDir}

############### Removing extended regions #############
echo 'Removing extended regions...'
unextend=${lrcstats}/src/preprocessing/unextend_alignments/unextend_alignments.py
unextendOutput=$outputDir/${testName}_unextended.maf
python $unextend -i $maf -m $unextendOutput
maf=$unextendOutput

############### Collecting data ###########
echo 'Collecting data...'

aligner=${lrcstats}/src/aligner/aligner
statsOutput=${outputDir}/${testName}.stats

$aligner stats -m $maf -o $statsOutput -p ${PBS_NUM_PPN}

input=${statsOutput}

############## Summarizing statistics ############
echo 'Summarizing statistics...'

summarizeStats=${lrcstats}/src/statistics/summarize_stats.py
statsOutput=${prefix}/stats/${program}/${testName}/${testName}_stats.txt

python ${summarizeStats} -i ${input} -b -o ${statsOutput}
```

Jabba:
```bash
set -e
mkdir -p ${outputDir}

############### Removing extended regions #############
echo 'Removing extended regions...'
unextend=${lrcstats}/src/preprocessing/unextend_alignments/unextend_alignments.py
unextendOutput=$outputDir/${testName}_unextended.maf
python $unextend -i $maf -m $unextendOutput
maf=$unextendOutput

############### Collecting data ###########
echo 'Collecting data...'

aligner=${lrcstats}/src/aligner/aligner
statsOutput=${outputDir}/${testName}.stats

$aligner stats -m $maf -o $statsOutput -t -p ${PBS_NUM_PPN}

input=${statsOutput}

############## Summarizing statistics ############
echo 'Summarizing statistics...'

summarizeStats=${lrcstats}/src/statistics/summarize_stats.py
statsOutput=${prefix}/stats/${program}/${testName}/${testName}_stats.txt

python ${summarizeStats} -i ${input} -o ${statsOutput}
```

LoRDEC:
```bash
set -e
mkdir -p ${outputDir}

############### Collecting data ###########
echo 'Collecting data...'

aligner=${lrcstats}/src/aligner/aligner
statsOutput=${outputDir}/${testName}.stats

$aligner stats -m $maf -o $statsOutput -p ${PBS_NUM_PPN}

input=${statsOutput}

############## Summarizing statistics ############
echo 'Summarizing statistics...'

summarizeStats=${lrcstats}/src/statistics/summarize_stats.py
statsOutput=${prefix}/stats/${program}/${testName}/${testName}_stats.txt

python ${summarizeStats} -i ${input} -b -o ${statsOutput}
```

Proovread:
```bash
set -e
mkdir -p ${outputDir}

############### Collecting data ###########
echo 'Collecting data...'

aligner=${lrcstats}/src/aligner/aligner
statsOutput=${outputDir}/${testName}.stats

$aligner stats -m $maf -o $statsOutput -t -p ${PBS_NUM_PPN}

input=${statsOutput}

############## Summarizing statistics ############
echo 'Summarizing statistics...'

summarizeStats=${lrcstats}/src/statistics/summarize_stats.py
statsOutput=${prefix}/stats/${program}/${testName}/${testName}_stats.txt

python ${summarizeStats} -i ${input} -o ${statsOutput}
```